### Pytorch implementaiton of Ising-2D simulation

In [ ]:
import torch
def ising2d_torch(N=20, 
                T=1.0, 
                J=1.0, 
                B=0.0, 
                n_steps=20000, 
                out_freq=10):
    '''
    Metropolis Monte Carlo simulator for the 2D Ising model using PyTorch.

    Parameters:
    spins (torch.Tensor): Initial spin configuration.
    T (float): Temperature.
    J (float): Interaction strength between spins.
    B (float): External magnetic field.
    n_steps (int): Number of Monte Carlo steps.
    out_freq (int): Output frequency for saving spin configurations, energy, and magnetization.
    device (str): Device to run the simulation ('cpu' or 'cuda').

    Returns:
    tuple: Arrays of spin configurations, energies, and magnetizations.
    '''
    
    device = torch.device("cpu")

    # Initialize spins
    spins = torch.randint(0, 2, (N, N), device=device) * 2 - 1
    
    M_t = spins.sum()
    neighbors  = spins.roll(1, dims=0) + spins.roll(1, dims=1) 
    E_t        = -J * (spins * neighbors).sum() + M_t*B
   
    S, E, M = [], [], []

    for step in range(n_steps):

        i, j = torch.randint(0, N, (2,), device=device)

        z = spins[(i + 1) % N, j] + spins[(i - 1) % N, j] + spins[i, (j + 1) % N] + spins[i, (j - 1) % N]

        dE = 2 * spins[i, j] * (J * z + B)
        dM = 2 * spins[i, j]

        if torch.rand(1, device=device) < torch.exp(-dE / T):
            spins[i, j] *= -1
            E_t += dE
            M_t += dM

        if step % out_freq == 0:
            S.append(spins.clone())
            E.append(E_t / N**2)
            M.append(M_t / N**2)

    return torch.stack(S), torch.tensor(E), torch.tensor(M)

In [ ]:
#Parameters
params = {'N':20,
          'J':1, 
          'B':0, 
          'T': 4,
          'n_steps': 10000, 
          'out_freq': 10}


S, E, M = ising2d_torch(**params)